# **Shooting Method with Numerov Algorithm to Solve the Time Independent Schrödinger Equation for 1D Quantum Well**

### [Go back to index](./index.ipynb)

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

<p style="text-align: justify;font-size:15px">  
    This notebook displays interactively the eigenfunctions (plotted at the height of the corresponding
    eigenvalues) for a single one-dimensional quantum well, as obtained from the numerical solutions.
    This notebook demonstrates using the shooting method with Numerov algorithm to search the
    eigenfunctions and eigenvalues. 
</p>

## **Numerov algorithm**

<p style="text-align: justify;font-size:15px"> 
The time independent Schrödinger equation (TISE) is:   
</p>

$\large \left[
  -\dfrac{\hslash^2}{2m} \, \dfrac{\partial^2}{\partial x^{2}} + V\right] \psi(x) = E\psi(x)$ (1)
  
<p style="text-align: justify;font-size:15px"> 
We can rewrite the TISE as:  
</p>

$\large  \dfrac{\partial^2}{\partial x^{2}} \psi(x) = -\dfrac{2m}{\hslash^2} \left[E-V\right]\psi(x)$ (2)

<p style="text-align: justify;font-size:15px"> 
    For one dimensional system, the second-derivative can be evaluated numerically. 
</p>

$\large \psi ''(x_{i})= \dfrac{1}{\delta x^2}\left[ \psi(x_{i+x})-2\psi(x_i)+\psi(x_{i-1}) \right]$ (3)

<p style="text-align: justify;font-size:15px"> 
    One can also include the 4th derivative to increase the accuracy.
</p>

$\large \psi ''(x_{i})= \dfrac{1}{\delta x^2}\left[ \psi(x_{i+x})-2\psi(x_i)+\psi(x_{i-1}) \right]- \dfrac{\delta x^2}{12} \psi(x_i)^{(4)} $ (4)

<p style="text-align: justify;font-size:15px">
    Substituting equation 4 or 3 into equation 2, one can solve the eigenfunctions iteratively 
    for any given eigenvalue E. However, the values of the first two starting points are unknown. 
    For the square well potential as shown below, we can assume $\psi(x_0)$ is zero and $\psi(x_1)$
    is a very small positive number.
</p>

## **Shooting method**

<p style="text-align: justify;font-size:15px"> 
    "In numerical analysis, the shooting method is a method for solving a boundary value problem by 
    reducing it to the system of an initial value problem. Roughly speaking, we 'shoot' out trajectories
    in different directions until we find a trajectory that has the desired boundary value."
</p>

https://en.wikipedia.org/wiki/Shooting_method

<p style="text-align: justify;font-size:15px">
    When numerically solving the equation from left to right, we need to make sure the boundary 
    condition is also fulfilled at the right edge, which is $\psi(x_{+\infty})=0$. As mentioned above, 
    one can try any value for the eigenvalue E. But only the true eigenvalue will lead to the solved
    wavefunction converge to zero at right side. Through scanning the eigenvalues and monitoring the
    solved function on the right edge, we can approach the true eigenfunction and eigenvalue. This 
    numerical method is called shooting method.
</p>

## **Instruction for using this notebook**

<p style="text-align: justify;font-size:15px">
    In the figure, the soild lines show the wavefunctions and their corresponding eigenvalues, 
    which are solved by matrix diagonalization. There is a dash red line at the bottom of the 
    figure, which shows the Numerov method calculated wavefunction with given eigenvalue E. 
</p>

<p style="text-align: justify;font-size:15px">
    There are four sliders to control the given eigenvalue E. The first slider controls the
    precision for tenths and hundreths. The second slider controls thousandths and ten thousandths
    decimal. The third slider controls hundred thousandths and millionths. The fourth slider
    controls ten millionths and hundred millionths. The full value is shown above the first 
    slider.
</p>

<p style="text-align: justify;font-size:15px">
    You need slowly move the 1st slider and observe the tail of the dash line. Once you 
    see the tail change directions (up or down), the true value is between these two values.
    You need go back to the smaller value and start to tun the 2nd slider. Then the 3rd and
    4th slider. When the absolute value at the left edge is smaller than 0.001, the dash line
    will turn to green. It reaches the desirable eigenwavefunction and its corresponding
    eigenvalue.
</p>

<p style="text-align: justify;font-size:15px">
    You can also use the "Auto search" button, which finds the most closest eigenvalue and 
    eigenfunction (search in upward direction). In order to compare, you may also need to 
    click the "Flip eigenfunctions" button.
</p>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [ ]:
%matplotlib widget

from numpy import linspace, sqrt, ones, arange, diag, argsort, zeros
from scipy.linalg import eigh_tridiagonal
import matplotlib.pyplot as plt


colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f'];
ixx = 0

def singlewell_potential(x, width, depth):
    x1 = zeros(len(x))
    for i in range(len(x)):
        if x[i] > - width/2.0 and x[i] < width/2.0:
            x1[i] = depth
    return x1


def diagonalisierung(hquer, L, N, pot=singlewell_potential, width = 0.1, depth = 0.0):
    """Calculated sorted eigenvalues and eigenfunctions. 

       Input:
         hquer: Planck constant
         L: set viewed interval [-L,L] 
         N: number of grid points i.e. size of the matrix 
         pot: potential function of the form pot
         x0: center of the quantum well
         width: the width of the quantum well
         depth: the depth of the quantum well
       Ouput:
         ew: sorted eigenvalues (array of length N)
         ef: sorted eigenfunctions, ef[:,i] (size N*N)
         x:  grid points (arry of length N)
         dx: grid space
         V:  Potential at positions x (array of length N)
    """
    x = linspace(-L, L, N+2)[1:N+1]                 # grid points
    dx = x[1] - x[0]                                # grid spacing
    V = pot(x, width, depth)
    z = hquer**2 /2.0/dx**2                         # second diagonals

    ew, ef = eigh_tridiagonal(V+2.0*z, -z*ones(N-1))
    ew = ew.real                                    # real part of the eigenvalues
    ind = argsort(ew)                               # Indizes f. sort. Array
    ew = ew[ind]                                    # Sort the ew by ind
    ef = ef[:, ind]                                 # Sort the columns 
    ef = ef/sqrt(dx)                                # Correct standardization 
    return ew, ef, x, dx, V


def plot_eigenfunktionen(ax, ew, ef, x, V, width=1, Emax=0.05, fak= 5.0):
    """Plot of the lowest eigenfunctions 'ef' in the potential 'V (x)'
       at the level of the eigenvalues 'ew' in the plot area 'ax'.
    """
    global lnum, lax1
    
    fak = fak/100.0;
    
    ax[0].axhspan(0.0, Emax, facecolor='lightgrey')
    
    ax[0].set_xlim([min(x), max(x)])
    ax[0].set_ylim([min(V)-0.05, Emax])
    
    ax[0].set_xlabel(r'$x/a$', fontsize = 10)
    ax[0].set_ylabel(r'$V(x)/V_0\ \rm{, Eigenfunctions\ with\ Eigenvalues}$', fontsize = 10)
    
    ax[1].set_xlim([min(x), max(x)])
    ax[1].set_ylim([min(V)-0.05, Emax])
    
    loop1.min = min(V)-0.03
    loop1.value = loop1.min
    loop2.value = loop2.min
    loop3.value = loop3.min
    loop4.value = loop4.min
    
    ax[1].yaxis.set_label_position("right")
    ax[1].yaxis.tick_right()
    
    ax[1].get_xaxis().set_visible(False)
    ax[1].set_ylabel(r'$\rm{\ Eigenvalues}$', fontsize = 10)
    
    indmax = sum(ew<=0.0)                       
    if not hasattr(width, "__iter__"):           
        width = width*ones(indmax)               
    for i in arange(indmax):                     
        ax[0].plot(x, fak*ef[:, i]+ew[i], linewidth=width[i]+.1, color=colors[i%len(colors)])
        ax[1].plot(x, x*0.0+ew[i], linewidth=width[i]+2.5, color=colors[i%len(colors)])
        
    ax[0].plot(x, V, c='k', linewidth=1.6)
    lnum, = ax[0].plot(x, x*0 + loop1.value,'r--', linewidth=1.0)
    lax1, = ax[1].plot(x, x*0 + loop1.value,'r--', linewidth=1.0)
    

In [ ]:
from ipywidgets import FloatSlider, jslink, VBox, HBox, Button, Label, Layout, Dropdown

mu = 0.06                                            # Potential parameter
L = 1.5                                              # x range [-L,L]
N = 200                                              # Number of grid points
hquer = 0.06                                         # Planck constant
sigma_x = 0.1                                        # Width of the Gaussian function
zeiten = linspace(0.0, 10.0, 400)                    # time


swidth = FloatSlider(value = 1.2, min = 0.1, max = 2.0, description = 'Width: ')
sdepth = FloatSlider(value = -0.2, min = -1.0, max = 0.0, step = 0.05, description = 'Depth: ')
sfak = FloatSlider(value = 5, min = 1.0, max = 10.0, step = 1.0, description = r'Zoom factor: ')


update = Button(description="Show all")
flip = Button(description="Flip eigenfunctions")
search = Button(description="Auto search")

order = Dropdown(options=['No', 'Yes'], value='No', 
                 description="With 4th derivative correction",
                 style={'description_width': 'initial'})


loop1 = FloatSlider(value = -0.25, min = -0.25, max = 0.00, description = '/1: ', 
                    layout=Layout(width='600px'), step = 0.01)
loop2 = FloatSlider(value = -100, min = -100, max = 0.0, description = '/10000:', 
                  layout=Layout(width='600px'), step =1.0, readout_format='.0f')
loop3 = FloatSlider(value = -100, min = -100, max = 0.0, description = '/1000000:', 
                  layout=Layout(width='600px'), step =1.0, readout_format='.0f')
loop4 = FloatSlider(value = -100, min = -100, max = 0.0, description = '/100000000:', 
                  layout=Layout(width='600px'), step =1.0, readout_format='.0f')

Leng = Label('')

width = 1.2
depth = -0.2
fak = 5.0

ew, ef, x, dx, V = diagonalisierung(hquer, L, N, width = width, depth = depth)
    
fig, ax = plt.subplots(1, 2, figsize=(7,5), gridspec_kw={'width_ratios': [10, 1]})
fig.canvas.header_visible = False
fig.canvas.layout.width = "750px"

fig.suptitle(r'Numerical Solution ($\psi$) of the Schrödinger Equation for 1D Quantum Well', fontsize = 12)
plot_eigenfunktionen(ax, ew, ef, x, V)

def Numerov(y, E, Vn, dxx):
    y = zeros(len(y));
    y[0] = 0.0;
    y[1] = 0.00000001;
    
    k2 = 2.0/(hquer**2)*(E-Vn)*dxx*dxx;
    
    for i in arange(2, len(y)):
        if order.value == "Yes":
            y[i] = (2*(12.0-5.0*k2[i-1])*y[i-1] - (12+k2[i-2])*y[i-2])/(12+k2[i]);
        elif order.value == "No":
            y[i] = 2*y[i-1] - k2[i-1]*y[i-1] - y[i-2]
    return y/(sqrt(abs(y)**2*dxx).sum())

def plot_numerov(c):
    Nn = 1000
    xx = linspace(-L, L, Nn+2)[1:Nn+1]
    dxx = xx[1] - xx[0];
    Vn = singlewell_potential(xx, width = swidth.value, depth = sdepth.value)
    yy = zeros(len(xx));
    
    Evalue = loop1.value + loop2.value/10000.0 + loop3.value/1000000.0 + loop4.value/100000000.0;
                   
    yy = Numerov(yy, Evalue, Vn, dxx);
    
    if abs(yy[-1]) < 0.001:
        lnum.set_color("green")
        lax1.set_color("green")
    else:
        lnum.set_color("red")
        lax1.set_color("red")

    Leng.value = "Current value: " + "{:.8f}".format(Evalue)    
    lnum.set_data(xx, yy + Evalue)
    lax1.set_data(xx, xx*0 + Evalue)
    
def on_auto_search(b):
    Nn = 1000
    xx = linspace(-L, L, Nn+2)[1:Nn+1]
    dxx = xx[1] - xx[0];
    Vn = singlewell_potential(xx, width = swidth.value, depth = sdepth.value)
    yy = zeros(len(xx));
    
    Evalue = loop1.value + loop2.value/10000.0 + loop3.value/1000000.0 + loop4.value/100000000.0;
    yy = Numerov(yy, Evalue, Vn, dxx);
    
    increment = 0.01
    while abs(yy[-1]) > 0.001:
        tail_old = yy[-1]
        Evalue += increment;
        yy = Numerov(yy, Evalue, Vn, dxx);
        tail_new = yy[-1]
        
        if tail_old*tail_new < 0:
            Evalue -= increment
            increment /= 100.0
            yy = Numerov(yy, Evalue, Vn, dxx);
            
    Leng.value = "Current value: " + "{:.8f}".format(Evalue)        
    lnum.set_data(xx, yy + Evalue)
    lax1.set_data(xx, xx*0 + Evalue)
    
    loop1.value = int(Evalue*100)/100.0;
    loop2.value = int((Evalue-loop1.value)*10000); 
    loop3.value = int((Evalue-loop1.value-loop2.value/10000)*1000000);
    loop4.value = int((Evalue-loop1.value-loop2.value/10000-loop3.value/1000000)*100000000)
    

def on_update_click(b):
    for i in ax[0].lines:
        i.set_alpha(1.0)
    for i in ax[1].lines:
        i.set_alpha(1.0)

def on_width_change(change):
    global ew, ef, x, dx, V
    ax[0].lines = []
    ax[1].lines = []
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass

    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, width = swidth.value, depth = sdepth.value)
    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value)

def on_depth_change(change):
    global ew, ef, x, dx, V
    ax[0].lines = []
    ax[1].lines = []
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass

    ew, ef, x, dx, V = diagonalisierung(hquer, L, N, width = swidth.value, depth = sdepth.value)
    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value)
    
def on_xfak_change(change):
    ax[0].lines = []
    ax[1].lines = []
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass

    plot_eigenfunktionen(ax, ew, ef, x, V, fak = sfak.value)

def on_press(event):
    global ann, ann1, ixx
    
    ixx = min(enumerate(ew), key = lambda x: abs(x[1]-event.ydata))[0]
    
    for i in range(len(ax[1].lines)-1):
        ax[0].lines[i].set_alpha(0.1)
        ax[1].lines[i].set_alpha(0.1)
        ax[0].lines[i].set_linewidth(1.1)
        
    ax[0].lines[ixx].set_alpha(0.5)
    ax[1].lines[ixx].set_alpha(0.5)
    ax[0].lines[ixx].set_linewidth(2.0)
    
    try:
        ann.remove()
        ann1.remove()
    except:
        pass
    
    ann = ax[0].annotate(s = 'n = ' + str(ixx+1), xy = (0, ew[ixx]), xytext = (-0.15, ew[ixx]), xycoords = 'data', color='k', size=15)
    ann1 = ax[1].annotate(s = str("{:.3f}".format(ew[ixx])), xy = (0, ew[ixx]), xytext = (-1.2, ew[ixx]+0.005), xycoords = 'data', color='k', size=9)

def on_flip_eigenfunctions(b):
    x = lnum.get_xdata();
    y = lnum.get_ydata();
    lnum.set_data(x, -y+2.0*y[0])

    
cid = fig.canvas.mpl_connect('button_press_event', on_press)

swidth.observe(on_width_change, names = 'value')
sdepth.observe(on_depth_change, names = 'value')
sfak.observe(on_xfak_change, names = 'value')

update.on_click(on_update_click)
flip.on_click(on_flip_eigenfunctions)
search.on_click(on_auto_search)

loop1.observe(plot_numerov, names = 'value')
loop2.observe(plot_numerov, names = 'value')
loop3.observe(plot_numerov, names = 'value')
loop4.observe(plot_numerov, names = 'value')

label1 = Label(value="(click on a state to select it)");
label2 = Label(value="(click to flip all eigenfunctions)");

display(Leng, loop1, loop2, loop3, loop4) 
display(HBox([order, search]), HBox([swidth, sdepth]), HBox([update, label1]), HBox([flip, label2]))

* **Width:** the width of the quantum well.
* **Depth:** the depth of the quantum well.
* **Zoom factor:** the zoom factor of the eigenfunctions. 